In [ ]:
import cv2
import os
from tqdm import tqdm

In [ ]:
for path, subdirs, files in os.walk('haarcascade/'):
    for name in tqdm(files) :
        video_path = os.path.join(path, name)
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        H, W, _ = frame.shape

        output_folder = "haarcascade cropped/" + video_path[:-4]
        os.makedirs(output_folder, exist_ok=True)

        if not cap.isOpened():
            print("Error: Gagal membuka video.")
            exit()

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        target_duration = 5
        video_count = 1

        current_duration = 0
        output_video = None
        frame_list = []

        while ret:
            if not ret:
                break

            frame_list.append(frame)
            current_duration += 1 / fps

            if current_duration >= target_duration:
                output_video = cv2.VideoWriter(f'{output_folder}/output_video_{video_count}.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, (W, H))
                for frame in frame_list:
                    output_video.write(frame)
                output_video.release()
                video_count += 1
                current_duration = 0
                frame_list = []

            ret, frame = cap.read()

        if current_duration > 0:
            output_video = cv2.VideoWriter(f'{output_folder}/output_video_{video_count}.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, (W, H))
            for frame in frame_list:
                    output_video.write(frame)
                    
            frames_to_add = int((target_duration - current_duration) * fps)
            for i in range(frames_to_add):
                output_video.write(frame_list[-1])
                
            output_video.release()
            video_count += 1
            current_duration = 0
            frame_list = []

        video_count = 0
        cap.release()